---
### Step 0: Load The Data

In [1]:
# Import libraries
import math
import cv2
import csv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split 
from keras.models import Sequential, model_from_json
from keras.utils import to_categorical
from keras.layers import Dense, Flatten, Lambda, Conv2D, Activation, Dropout
from keras.callbacks import ModelCheckpoint
%matplotlib inline

Using TensorFlow backend.


In [2]:
#img_data_set = []
#label_data_set = []
lines = []
with open('labeled_images.csv') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
        for line in reader:
            lines.append(line)
print(len(lines))

4110


In [3]:
#labeled_pictures = pd.read_csv('labeled_images.csv')
shuffle(lines)
train_data, val_data = train_test_split(lines, test_size = 0.2)
print(len(train_data))
print(len(val_data))

3288
822


---
### Step 1: Define Data Generator

In [6]:
def data_generator (data_lines, batch_size = 40) :
    num_lines = len(data_lines)
    while True:
        shuffle(data_lines)
        for offset in range(0, num_lines, batch_size):
            batch_lines = data_lines[offset:offset + batch_size]
            images = []
            labels = []
            for batch_line in batch_lines:
                img = cv2.cvtColor(cv2.imread('./rec_images/'+str(batch_line[0])+'.jpeg'), cv2.COLOR_BGR2HSV)
                resized_image = cv2.resize(img, (400, 300))
                images.append(resized_image)
                labels.append(batch_line[1])
            X = np.array(images)
            y = to_categorical(np.array(labels), num_classes=5)
            yield shuffle(X, y)

---
### Step 2: Define Model

In [7]:
model = Sequential()
#normalize data
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(300,400,3)))

model.add(Conv2D(24,(5,5), strides=(2,2), activation='relu'))
model.add(Conv2D(36,(5,5), strides=(2,2), activation='relu'))
model.add(Conv2D(48,(5,5), strides=(2,2), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
model.fit_generator(generator = data_generator(train_data),
                    validation_data = data_generator(val_data),
                    epochs = 10,
                    steps_per_epoch  = math.ceil(len(train_data) / 128),
                    validation_steps = math.ceil(len(val_data)   / 128)    )
model.save('model.h5')
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
print('Model saved')

Epoch 1/10
26/26 [==============================] - 71s 3s/step - loss: 1.2842 - acc: 0.4663 - val_loss: 1.0064 - val_acc: 0.6107
Epoch 2/10
26/26 [==============================] - 71s 3s/step - loss: 0.9267 - acc: 0.6279 - val_loss: 0.7931 - val_acc: 0.7500
Epoch 3/10
26/26 [==============================] - 76s 3s/step - loss: 0.7324 - acc: 0.7356 - val_loss: 0.7239 - val_acc: 0.7710
Epoch 4/10
26/26 [==============================] - 73s 3s/step - loss: 0.6673 - acc: 0.7365 - val_loss: 0.5444 - val_acc: 0.7893
Epoch 5/10
26/26 [==============================] - 69s 3s/step - loss: 0.4630 - acc: 0.7904 - val_loss: 0.4025 - val_acc: 0.8250
Epoch 6/10
26/26 [==============================] - 69s 3s/step - loss: 0.4359 - acc: 0.8202 - val_loss: 0.3544 - val_acc: 0.8664
Epoch 7/10
26/26 [==============================] - 67s 3s/step - loss: 0.3115 - acc: 0.8729 - val_loss: 0.2882 - val_acc: 0.8964
Epoch 8/10
26/26 [==============================] - 69s 3s/step - loss: 0.2182 - acc: 0.91

In [12]:
test_image = cv2.cvtColor(cv2.imread('./rec_images/5822.jpeg'), cv2.COLOR_BGR2HSV)
resized_image = cv2.resize(test_image, (400, 300))
resized_image = np.array([resized_image])
pred_vects = model.predict(resized_image)
print("Predicted Probabilities: {}".format(pred_vects))
pred_labels = pred_vects.argmax(axis=-1)
print("Predicted Labels: {}".format(pred_labels))

Predicted Probabilities: [[2.5236413e-05 1.6024712e-03 9.9814630e-01 1.0115817e-15 2.2598584e-04]]
Predicted Labels: [2]
